In [ ]:
import base
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

## Setup

- Load data
- Add metadata
- Draw gates
- Gate transfected cells

Result from this section: DataFrame `df` representing transfected cells.

Load all transfection data collected as of 2024.04.11

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'

exp90_path = base_path/'2024.03.31_exp90'/'export'
exp90_2_path = base_path/'2024.04.02_exp90.2'/'export'
exp90_3_path = base_path/'2024.04.02_exp90.3'/'export'
exp90_4_path = base_path/'2024.04.05_exp90.4'/'export'

exp91_path = base_path/'2024.04.08_exp91'/'export'

plates = pd.DataFrame({
    'data_path': [exp90_path/'plate1', exp90_path/'plate2', exp90_2_path, exp90_3_path/'plate1', exp90_3_path/'plate2', exp90_4_path,
                  exp91_path/'plate1.1', exp91_path/'plate1.2', exp91_path/'plate1.3', exp91_path/'plate2.1', exp91_path/'plate2.2', exp91_path/'plate2.3',],
    'yaml_path': ([exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', exp90_path/'exp90_plate2_wells.yaml',] +
                  [exp91_path/'exp91_plate1_wells.yaml']*3 + [exp91_path/'exp91_plate2.1_wells.yaml', exp91_path/'exp91_plate2.2_wells.yaml', exp91_path/'exp91_plate2.3_wells.yaml']
                  ),
    'biorep': [1, 1, 2, 3, 3, 2,
               1, 2, 3, 1, 2, 3,
               ],
    'exp': ['exp90', 'exp90', 'exp90.2', 'exp90.3', 'exp90.3', 'exp90.4',
            'exp91', 'exp91', 'exp91', 'exp91', 'exp91', 'exp91',
            ]
})

output_path = rd.rootdir/'output'/'circuit-tuning'
cache_path = rd.rootdir/'output'/'circuit-tuning'/'data.gzip'

metadata_keys = set()
for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)
    metadata_keys.update(rd.flow.load_well_metadata(p).keys())
display(metadata_keys)

In [ ]:
# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    data = rd.flow.load_groups_with_metadata(plates)

    # Remove unneeded columns and negative channel values
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    for c in data.columns:
        if not(c in (list(plates.columns) + channel_list + list(metadata_keys))):
            data.drop(columns=c, inplace=True)
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')
display(data)

# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
construct_palette = metadata_dict['color']
construct_markers = metadata_dict['markers']

Draw gates based on untransfected population, then gate transfected cells

In [ ]:
gates = pd.DataFrame()
channel_list = ['tagBFP-A', 'mGL-A', 'mCherry-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Add missing gates
gates.loc[gates['exp']=='exp90.4', channel_list] = gates.loc[gates['exp']=='exp90.2', channel_list].values

# Indicate which channels are relevant for each experiment
gates.sort_values(['exp'], inplace=True)
gates['marker'] = 'mGL-A'
gates['output'] = 'mRuby2-A'

display(gates)

Gate data per experiment based on transfection marker expression

In [ ]:
data = data.groupby('exp')[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
max = 1e6
df = data[(data['expressing']) & (data['output']<max)]
display(df)

### Explore miR-FF5 data

In [ ]:
ff5_df = df[(df['promoter']=='EF1a') & 
            ((df['group']=='base') | # no miR + no TS
             ((df['group'].isin(['ts3', 'ts5'])) & (df['ts'].isin(['FF5x1', 'FF6x1']))) | # TS only
             ((df['group'].isin(['miR','controller'])) & (df['miR']=='miR.FF5')) # miR only, controller
            )]
ff5_df.sort_values('design', inplace=True)
group_order = {'base': 1, 'miR': 2, 'ts3': 3, 'ts5': 4, 'controller': 5}
ff5_df.sort_values('group', key=lambda x: x.map(group_order), inplace=True)
ff5_df.sort_values('ts_kind', inplace=True)
display(ff5_df)

In [ ]:
plot_df = ff5_df[(ff5_df['biorep']==3)]
g = sns.displot(data=plot_df, x='output', hue='construct', palette=construct_palette, col='group', kind='kde', col_wrap=4,
                facet_kws=dict(margin_titles=True, sharex=True, sharey=True), log_scale=True, legend=False, common_norm=False,)
g.figure.savefig(rd.outfile(output_path/(f'hist_constructs.svg')), bbox_inches='tight')

In [ ]:
plot_df = ff5_df
g = sns.displot(data=plot_df[~plot_df['group'].isin(['base','controller'])], x='output', hue='construct', 
                palette=construct_palette, row='biorep', col='group', kind='kde', log_scale=True, legend=False, 
                common_norm=False, facet_kws=dict(margin_titles=True, sharex=True, sharey=True), )
for (biorep, group), ax in g.axes_dict.items():
    sns.kdeplot(data=plot_df[(plot_df['group']=='base') & (plot_df['biorep']==biorep)], x='output', 
                log_scale=True, legend=False, common_norm=False, ax=ax, color='black', ls=':')
g.figure.savefig(rd.outfile(output_path/(f'hist_FF5_controls.svg')), bbox_inches='tight')